# Рекомендация тарифов

**Описание проектной работы**

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Оператор хочет построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

Предоставлены данные о поведении клиентов, которые уже перешли на эти тарифы.

**Цель проекта**

Требуется построить модель для задачи классификации, которая предложит подходящий тариф. Итоговая `accuracy` должна быть не ниже `0.75`. 

**План выполнения работы**
   
1. Изучение общей информации
2. Разделение данных на выборки
3. Исследование моделей
4. Проверка модели на тестовой выборке
5. Проверка модели на адекватность
6. Общие выводы

# Содержание

[1. Изучение общей информации](#Изучение-общей-информации)

[2. Разделение данных на выборки](#Разделение-данных-на-выборки)

[3. Исследование моделей](#Исследование-моделей)

[4. Проверка модели на тестовой выборке](#Проверка-модели-на-тестовой-выборке)

[5. Проверка модели на адекватность](#Проверка-модели-на-адекватность)

[6. Общие выводы](#Общие-выводы)

## Изучение общей информации

[Назад к содержанию](#Содержание)

In [1]:
# импорт библиотек:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [2]:
# чтение исходного датасета:

df = pd.read_csv('/datasets/users_behavior.csv')
print('Датафрейм сформирован!')

Датафрейм сформирован!


In [3]:
# выведем несколько строк исходного датафрейма:

df.head()


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
# выведем общую информацию о датафрейме:

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
# посмотрим на датафрейм с помощью метода describe:

df.describe()


,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [6]:
# проверка наличия пропусков:

df.isna().sum()


calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [7]:
# поиск дубликатов:

df.duplicated().sum()


0

**Выводы по изучению общей информации**

1. Загрузили и преобразовали исходный датасет в датафрейм. Присвоили переменной `df` исходный датафрейм;
2. Вывели несколько строк датафрейма;
3. Изучили общую информацию о датафрейме:

  - датафрейм состоит из `5` столбцов и `3214` строк;
  - типы данных в столбцах не требуют преобразования в другие типы и готовы для работы;
  - пропуски в данных отсутствуют;
  - явные дубликаты отсутствуют.
  
4. Описание данных. Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
      
  - `сalls` — количество звонков,
  - `minutes` — суммарная длительность звонков в минутах,
  - `messages` — количество sms-сообщений,
  - `mb_used` — израсходованный интернет-трафик в Мб,
  - `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).
      
Исходный датафрейм полностью готов для дальнейшей работы.


## Разделение данных на выборки

[Назад к содержанию](#Содержание)

Создадим переменные для признаков и целевого признака: 

   - в качестве целевого признака `target` будем использовать значения из столбца `is_ultra` - `0` или `1`;
   - в качестве признаков `features` будем использовать все признаки, за исключением целевого. 
   
Пропишем переменные:

In [8]:
# признаки: 
features = df.drop('is_ultra', axis=1)

# целевой признак:
target = df['is_ultra']

Поскольку у нас отсутствует отдельный датасет для тестирования модели, разобьем исходный датафрейм на обучающую, валидационную и тестовую выборки в соотношении 3:1:1:

   - 60% под обучающую выборку;
   - 20% под валидационную выборку;
   - 20% под тестовую выборку 

В качестве параметра `random_state` примем значение `12345`.

Разделение выборок выполним поэтапно: сперва выделим 60% под обучающую выборку, оставшиеся 40% поделим поровну между валидационной и тестовой выборками.


В связи с тем, что наш исходный датасет небольшой, то имеет смысл разделять данные с помощью стратификации. Стратификация позволяет избежать случая когда распределение тренировочного и тестового множеств оказываются не таким, как у всего исходного множества. Т.е. стратификация помогает сохранить соотношение классов, представленное в исходном датасете.


In [9]:
# выделим 60% под обучающую выборку
# применим аргумент stratify  к целевому признаку target:

features_train, features_valid_test, target_train, target_valid_test = \
    train_test_split(features, target, test_size = 0.4, random_state = 12345, stratify = target)


In [10]:
# проверим количество получившихся объектов:

print('features_train:', features_train.shape)
print('features_valid_test:', features_valid_test.shape)
print('target_train:', target_train.shape)
print('target_valid_test:', target_valid_test.shape)


features_train: (1928, 4)
features_valid_test: (1286, 4)
target_train: (1928,)
target_valid_test: (1286,)


1928 строк - это 60% от общего числа строк в исходном датафрейме (3214), т.е. разбили верно. 

Далее оставшиеся 40% разобъем поровну между валидационной и тестовой выборками.


In [11]:
# выделим по 20% под валидационную и тестовую выборки
# аргумент stratify применим к target_valid_test:

features_valid, features_test, target_valid, target_test = \
    train_test_split(features_valid_test, target_valid_test, \
    test_size = 0.5, random_state = 12345, stratify = target_valid_test) 
                                                    

In [12]:
# проверим количество получившихся объектов:

print('features_valid:', features_valid.shape)
print('features_test:', features_test.shape)
print('target_valid:', target_valid.shape)
print('target_test:', target_test.shape)


features_valid: (643, 4)
features_test: (643, 4)
target_valid: (643,)
target_test: (643,)


643 строки - это 20% от общего числа строк в исходном датафрейме. Т.е. разбили верно.

**Вывод по разделению данных на выборки**

В данном пункте мы разбили исходный датафрейм на обучающую, валидационную и тестовую выборки в соотношении `3:1:1`:

   - `60%` выделили под обучающую выборку: `features_train` и `target_train;
   - `20%` под валидационную выборку: `features_valid` и `target_valid`;
   - `20%` под тестовую выборку: `features_test` и `target_test`


## Исследование моделей

[Назад к содержанию](#Содержание)

Целевой признак, который нужно предсказать в нашей задаче, это значение `0` или `1` в столбце `is_ultra`. Целевой признак является категориальным, т.к. принимает всего два значения: `0` или `1`. Поэтому наша задача сводится к бинарной (двоичной) классификации.

Решить задачу можно следующими способами:

   - решающее дерево;
   - случайный лес;
   - логистическая регрессия
   

### Исследование модели с помощью решающего дерева


Обучим модель с помощью решающего дерева. С помощью цикла будем изменять значения гиперпараметра `max_depth` от 1 до 5. Для каждого значения `max_depth` выведем значение качества (`accuracy`) на валидационной выборке.

In [14]:
# обучение модели методом DecisionTreeClassifier
# изменение гиперпараметра max_depth от 1 до 5:
   
best_model = None
best_result = 0
max_depth = 0
for depth in range(1, 6):  
    model_DTC = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучение модели с заданным параметром max_depth
    model_DTC.fit(features_train, target_train) # обучение модели на тренировочной выборке
    result = model_DTC.score(features_valid, target_valid) # качество модели на валидационной выборке
    if result > best_result:
        best_model_DTC = model_DTC # сохранение наилучшей модели
        best_result = result #  сохранение наилучшего значения метрики accuracy на валидационных данных
        max_depth = depth

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Гиперпараметр max_depth =", max_depth)



Accuracy наилучшей модели на валидационной выборке: 0.7853810264385692
Гиперпараметр max_depth = 5


Обучили модель с помощью решающего дерева. Протестировали модель на валидационной выборке. Получили наилучшее значение качества модели `accuracy` равное `0.785` при значении гиперпараметра `max_depth` = `5`.


### Исследование модели с помощью случайного леса

Далее обучим модель с помощью случайного леса. Число деревьев возьмем от 1 до 30 и выберем наилучшую модель.


In [15]:
# обучение модели методом RandomForestClassifier
# изменение числа деревьев от 1 до 30

best_model = None
best_result = 0
estimators = 0
for est in range(1, 31):
    model_RFC = RandomForestClassifier(random_state=12345, n_estimators=est) # обучение модели с заданным количеством деревьев
    model_RFC.fit(features_train, target_train) # обучение модели на тренировочной выборке
    result = model_RFC.score(features_valid, target_valid) # качество модели на валидационной выборке
    if result > best_result:
        best_model_RFC = model_RFC # сохранение наилучшей модели
        best_result = result # наилучшее значение метрики accuracy на валидационных данных
        estimators = est # количество деревьев
        
print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Количество деревьев равно:", estimators)

Accuracy наилучшей модели на валидационной выборке: 0.7993779160186625
Количество деревьев равно: 21


Обучили модель с помощью случайного леса. Протестировали модель на валидационной выборке. Получили наилучшее значение качества модели `accuracy` равное `0.799` при количестве деревьев равное `21`.


### Исследование модели с помощью логистической регрессии

Обучим модель с помощью логистической регрессии. 

В качестве гиперпараметров примем:

   - `random_state` = `12345`;
   - алгоритм обучения будем использовать один из самых распространенных - 'lbfgs' — он подходит для решения большинства задач;
   - максимальное количество итераций `max_iter` примем равное `1000`.

In [16]:
# обучение модели с помощью логистической регрессии:

model_LgR = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_LgR.fit(features_train, target_train) # обучение модели на тренировочной выборке

model_LgR.score(features_valid, target_valid) # качество модели на валидационной выборке



0.7387247278382582

Обучили модель с помощью логистической регрессии. Получили самое низкое качество модели `0.738`.


### Вывод

Обучили модели на тренировочной выборке тремя способами:

   - с помощью дерева решений;
   
   - с помощью случайного леса;
   
   - с помощью логистической регрессии.
   
   
Проверили качество получившихся моделей на валидационной выборке. Получили следующие результаты:


   - наилучшее значение `accuracy` модели, обученной с помощью `дерева решений`, получили равное `0.785`;
   
   - наилучшее значение `accuracy` модели, обученной с помощью `случайного леса`, получили равное `0.799`, при количестве деревьев равное `21`;
   
   - значение `accuracy` модели, обученной с помощью `логистической регрессии`, получили равное `0.738`.

Как видно, самое `высокое` качество у модели, обученной с помощью `случайного леса`. Качество модели равно `0.799`.

Самое `низкое` качество у `логистической регрессии`: значение `accuracy` = `0.738`.


## Проверка модели на тестовой выборке

[Назад к содержанию](#Содержание)

Проверим наилучшую модель на тестовой выборке. 

In [17]:
# проверка наилучшей модели на тестовой выборке:

best_model_RFC.predict(features_test) # предсказание модели по тестовой выборке
best_model_RFC.score(features_test, target_test) # качество модели на тестовой выборке

0.80248833592535

Попробуем улучшить качество нашей наилучшей модели `best_model_RFC`. Для чего объединим обучающую и валидационную выборки в одну и заново обучим на ней нашу модель `best_model_RFC` с теми же гиперпараметрами. 

Тестировать новую модель будем также на тестовой выборке.


In [18]:
# перед объединением выборок посмотрим на них:
# features_train, features_valid, target_train и target_valid:

print('features_train:', features_train.shape)
print('features_valid:', features_valid.shape)
print('target_train:', target_train.shape)
print('target_valid:', target_valid.shape)

features_train: (1928, 4)
features_valid: (643, 4)
target_train: (1928,)
target_valid: (643,)


In [19]:
# объединение выборок features_train и features_valid:

train_df = pd.concat([features_train, features_valid])

# объединение выборок target_train и target_valid:

target_df = pd.concat([target_train, target_valid])

In [20]:
# проверим получившиеся выборки:

print(train_df.shape)
print(target_df.shape)

(2571, 4)
(2571,)


1928 + 643 = 2571, т.е. объединили верно

In [21]:
# обучение модели по обучающим большим выборкам train_df и target_df:

new_model = best_model_RFC.fit(train_df, target_df)

In [22]:
# проверка новой модели на тестовой выборке:

new_model.score(features_test, target_test)

0.8289269051321928

Действительно получилось увеличить качество модели. В связи с чем будем считать модель `new_model` итоговой моделью.

**Вывод после проверки наилучшей модели на тестовой выборке**

Сперва проверили наилучшую модель на тестовой выборке. Значение `accuracy` на тестовой выборке получили равным `0.802`.

Затем заново обучили модель на выборке побольше - объединили обучающую и валидационную выборки. После чего протестировали на тестовой выборке. Значение `accuracy` итоговой модели получили равным `0.828`. 


## Проверка модели на адекватность

[Назад к содержанию](#Содержание)

Чтобы оценить адекватность модели в задачах классификации, нужно сравнить её со случайной моделью. Для чего импортируем из библиотеки `sklearn DummyClassifier` - это классификатор, который делает прогнозы, используя простые правила.

В качестве параметра выберем `most_frequent` - этот параметр всегда предсказывает наиболее часто встречающуюся метку в обучающем наборе.


In [23]:
# импорт DummyClassifier  
from sklearn.dummy import DummyClassifier

# создание случайной dummy-модели
dc_mf = DummyClassifier(strategy="most_frequent") 

# обучение dummy-модели на тренировочной выборке:
dc_mf.fit(features_train, target_train)

dc_mf.predict(features_test) # предсказание dummy-модели по тестовой выборке
dc_mf.score(features_test, target_test) # качество dummy-модели на тестовой выборке


0.6936236391912908

`Accuracy` `dummy-модели` получили `0.693`, что гораздо ниже нашей итоговой модели - `0.828`.

Таким образом, подтвериди, что наша модель лучше случайной `dummy-модели`.

**Вывод проверки модели на адекватность**
   
Сравнили наилучшую модель со случайной. Наша модель лучше случайной. Значения качества нашей и случайной моделей `0.828` и `0.693`, соответственно.

## Общие выводы

[Назад к содержанию](#Содержание)

Целью настоящей работы было разработать систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: `«Смарт»` или `«Ультра»`. Для чего была предоставлена информация о поведении пользователей за месяц. В исходном датасете были предоставлены следующие данные:

   - количество звонков;
   - суммарная длительность звонков в минутах;
   - количество sms-сообщений;
   - израсходованный интернет-трафик в Мб;
   - каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).
   
Задача свелась к бинарной (двоичной) классификации. В результате проведения работы должны были обучить модель, способную по переданным ей признакам предсказать целевой признак - тариф `"Ультра"` или `"Смарт"`.   

В первую очередь разбили исходный датафрейм на следующие выборки:

   - 60% под обучающую выборку;
   - 20% под валидационную выборку;
   - 20% под тестовую выборку

В ходе проведения работы обучили три модели различными способами и получили различные качественные параметры моделей. Обучали модели на обучающей выборке, тестировали на валидационной выборке. Получили следующие результаты для моделей:

   - `решающее дерево` со значением `accuracy` = `0.785`;
   - `случайный лес` со значением `accuracy` = `0.799`;
   - `логистическая регрессия` со значением `accuracy` = `0.738`.
   
Исходя из результатов показателей качества, выбрали наилучшую модель, обученную с помощью `случайного леса` и значением `accuracy` = `0.799`.

С помощью выбранной модели проверили модель на тестовой выборке. Сперва получили значение `accuracy` равное `0.802`. Затем улучшили качество модели следующим способом: объединили обучающую и валидационную выборки в одну большую и заново обучили модель. Протестировали получившуюся модель на тестовой выборке и получили качество `итоговой` модели `accuracy` равное `0.828`.

Сравнили выбранную модель со случайной `dummy-моделью`. В результате `accuracy` `dummy-модели` получили `0.693`, что ниже нашей выбранной модели. В связи с чем можем считать нашу итоговую модель рабочей.
 